# Use DCP as initializer
---
## Define dataset

In [1]:
import numpy as np
from dcp import train, test
from data import SceneNet
from torch.utils.data import DataLoader
import sys
sys.path.append("dcp-master")
from model import DCP
import torch
import warnings
warnings.filterwarnings('ignore')


trainDataset = SceneNet(1024, "train")
valDataset = SceneNet(1024, "val")
train_loader = DataLoader(trainDataset, batch_size=32, shuffle=False, drop_last=False)
test_loader = DataLoader(valDataset, batch_size=1, shuffle=False, drop_last=False)
print(len(trainDataset))
print(len(valDataset))

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
299
299


## Define parameters

In [2]:
args = {
        "model_path": 'checkpoints/dcp_v1/models/model.best.t7',
        # "model_path": 'dcp-master/pretrained/dcp_v2.t7',
        "exp_name":"dcp_v1",
         "model":"dcp", 
         "emb_nn":"dgcnn", 
         "pointer":"identity", 
         "head":"svd", 
         "eval": True,
         'emb_dims': 512,
         'cycle': False,
         'use_sgd': False,
         'lr': 0.001,
         'epochs': 100,
         'n_blocks': 1,
         'dropout': 0.0,
         'ff_dims': 1024,
         'n_heads': 4,
         'use_sgd': False,
        }

## DCP Training

In [3]:
net = DCP(args)
train(args, net, train_loader, test_loader)

Use Adam


/root/miniconda3/envs/ml3d/lib/python3.9/site-packages/torch/optim/lr_scheduler.py:131: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  warnings.warn("Detected call of `lr_scheduler.step()` before `optimizer.step()`. "
100%|██████████| 299/299 [00:02<00:00, 143.15it/s]


Epoch 0: -train- Loss: 0.251428,  -val- Loss: 0.086488


100%|██████████| 299/299 [00:02<00:00, 146.10it/s]


Epoch 1: -train- Loss: 0.203835,  -val- Loss: 0.180736


...

Epoch 7: -train- Loss: 0.079790,  -val- Loss: 0.290948


100%|██████████| 299/299 [00:02<00:00, 148.24it/s]


Epoch 8: -train- Loss: 0.065057,  -val- Loss: 0.252361


100%|██████████| 299/299 [00:01<00:00, 159.55it/s]


Epoch 9: -train- Loss: 0.056748,  -val- Loss: 0.219683


## DCP Testing

In [4]:
net.load_state_dict(torch.load(args['model_path']), strict=False)

r, t = test(args, net, test_loader)
print(r[0])
print(t[0])

100%|██████████| 299/299 [00:02<00:00, 146.97it/s]


==FINAL TEST==
A--------->B
EPOCH:: 1, Loss: 0.086488, Cycle Loss: 0.000000, MSE: 2.036176, RMSE: 1.426947, MAE: 1.032615, rot_MSE: 32.801849, rot_RMSE: 5.727290, rot_MAE: 4.028041, trans_MSE: 0.081052, trans_RMSE: 0.284697, trans_MAE: 0.206162
B--------->A
EPOCH:: 1, Loss: 0.086488, MSE: 2.036176, RMSE: 1.426947, MAE: 1.030770, rot_MSE: 32.853836, rot_RMSE: 5.731827, rot_MAE: 4.051284, trans_MSE: 0.080918, trans_RMSE: 0.284460, trans_MAE: 0.204002
[[ 0.9988705   0.04650864 -0.0096752 ]
 [-0.04713354  0.9957025  -0.07971624]
 [ 0.00592625  0.08008248  0.99677086]]
[-0.4117757  -0.00475568  0.47055626]


## Visualization

In [5]:
import matplotlib.pyplot as plt
import k3d
from util import transform_point_cloud

def visualize_pointcloud(point_cloud1, point_size, point_cloud2=None, flip_axes=False, name='point_cloud', R=None, t=None):
    plot = k3d.plot(name=name, grid_visible=False, grid=(-0.55, -0.55, -0.55, 0.55, 0.55, 0.55))
    plt_points1 = k3d.points(positions=point_cloud1, point_size=point_size, color=0xd0d0d0)
    plot += plt_points1
    plt_points1.shader = '3d'
    if point_cloud2 is not None:
        plt_points2 = k3d.points(positions=point_cloud2, point_size=point_size, color=0x0dd00d)
        plot += plt_points2
        plt_points2.shader = '3d'
    plot.display()

def transform(point_cloud, R=None, t=None):
    t_broadcast = np.broadcast_to(t[:, np.newaxis], (3, point_cloud.shape[0]))
    return (R @ point_cloud.T + t_broadcast).T


src, target, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba = valDataset[20]
print(rotation_ab)
print(translation_ab)
# print(points1.shape)
transformed_src = transform_point_cloud(torch.tensor(src), torch.tensor(r[20]).unsqueeze(0), torch.tensor(t[20]).unsqueeze(0)).T
transformed_src2 = transform_point_cloud(torch.tensor(src), torch.tensor(rotation_ab).unsqueeze(0), torch.tensor(translation_ab).unsqueeze(0)).T
# transformed_src1 = transform_point_cloud(torch.tensor(src), r1, t1).T
# visualize_pointcloud(target.T, .03, transformed_src1)
visualize_pointcloud(target.T, .03, transformed_src)
visualize_pointcloud(target.T, .03, transformed_src2)
visualize_pointcloud(src.T, .03, target.T)

[[ 0.99980277 -0.00443393  0.0193574 ]
 [ 0.00360092  0.9990747   0.04285804]
 [-0.01952951 -0.04277989  0.9988936 ]]
[-0.0209675   0.00747702  0.02584793]


/tmp/ipykernel_486824/665646376.py:25: UserWarning: The use of `x.T` on tensors of dimension other than 2 to reverse their shape is deprecated and it will throw an error in a future release. Consider `x.mT` to transpose batches of matricesor `x.permute(*torch.arange(x.ndim - 1, -1, -1))` to reverse the dimensions of a tensor. (Triggered internally at  ../aten/src/ATen/native/TensorShape.cpp:2981.)
  transformed_src = transform_point_cloud(torch.tensor(src), torch.tensor(r[20]).unsqueeze(0), torch.tensor(t[20]).unsqueeze(0)).T


Output()

Output()

Output()

# ICP Calibration

In [6]:
from icp import test

valDataset = SceneNet(1024, "val", icp=True, r=r, t=t)
print(len(valDataset))
r_icp, t_icp = test(valDataset)

299


100%|██████████| 299/299 [21:46<00:00,  4.37s/it]

==FINAL TEST==
A--------->B
EPOCH:: -1, Loss: 0.302521, Cycle Loss: 0.000000, MSE: 0.036728, RMSE: 0.191645, MAE: 0.057405, rot_MSE: 96.504996, rot_RMSE: 9.823696, rot_MAE: 4.909005, trans_MSE: 0.286896, trans_RMSE: 0.535627, trans_MAE: 0.258043
B--------->A
EPOCH:: -1, Loss: 0.302521, MSE: 0.036728, RMSE: 0.191645, MAE: 0.056215, rot_MSE: 96.557699, rot_RMSE: 9.826378, rot_MAE: 4.932003, trans_MSE: 0.291971, trans_RMSE: 0.540344, trans_MAE: 0.260823


## Final Visualisation

In [7]:
src, target, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba = valDataset[20]
print(rotation_ab)
print(translation_ab)
# print(points1.shape)
transformed_src = transform_point_cloud(torch.tensor(src).double(), torch.tensor(r_icp[20]).unsqueeze(0).double(), torch.tensor(t_icp[20]).unsqueeze(0).double()).T
transformed_src2 = transform_point_cloud(torch.tensor(src), torch.tensor(rotation_ab).unsqueeze(0), torch.tensor(translation_ab).unsqueeze(0)).T
# transformed_src1 = transform_point_cloud(torch.tensor(src), r1, t1).T
# visualize_pointcloud(target.T, .03, transformed_src1)
visualize_pointcloud(target.T, .03, transformed_src)
visualize_pointcloud(target.T, .03, transformed_src2)
visualize_pointcloud(src.T, .03, target.T)

[[ 0.99980277 -0.00443393  0.0193574 ]
 [ 0.00360092  0.9990747   0.04285804]
 [-0.01952951 -0.04277989  0.9988936 ]]
[-0.0209675   0.00747702  0.02584793]


Output()

Output()

Output()